In [ ]:
import pandas as pd

import os
#import re

In [ ]:
# path to project: Edit here to point to your Drive
drive = './drive/MyDrive/'
project_dir = 'Colab Notebooks/IronHacks2024_Data_Analytics/DA_Week_8-9_Final_Project'
COLAB_PROJECT_PATH = drive+project_dir+'/Final_Project/'

In [ ]:
# Mount Google Drive
using_colab = False
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    # Mount Google Drive
    drive.mount('/content/drive', force_remount=True)
    using_colab = True
print(COLAB_PROJECT_PATH)
if using_colab and os.path.exists(COLAB_PROJECT_PATH):
    print('found colab path; redirecting to main project directory')
    os.chdir(COLAB_PROJECT_PATH)

Mounted at /content/drive
./drive/MyDrive/Colab Notebooks/IronHacks2024_Data_Analytics/DA_Week_8-9_Final_Project/Final_Project/
found colab path; redirecting to main project directory


In [ ]:
os.listdir('.')

['Scraper',
 'vgsales_current.csv',
 'Partial_Files_Prior_Merge',
 'Video_Games_Clean_EDA.ipynb',
 'Scrape_missing_data',
 'vgsales_critic_user.csv']

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from vgchartz_scraper import scrape_vg_chartz

In [ ]:
progress_tracker = './Scraper/Loaded_pages/track_pages.txt'

In [ ]:
progress_tracker.split('.')

['', '/Scraper/Loaded_pages/track_pages', 'txt']

In [ ]:
os.path.exists(os.path.dirname(progress_tracker))

True

In [ ]:
def page_formatted_file(part_num):
    t_file, ext = progress_tracker.split('.')[1:]
    part_suffix = '_' + f'{part_num}' if part_num else ''
    full_path = f'./{t_file}{part_suffix}.{ext}'
    return full_path

def create_if_not_exists(part_num):
    full_file = page_formatted_file(part_num)

    if not os.path.exists(full_file):
        with open(full_file, 'w') as file:
            file.write('')

In [ ]:
def record_progress(part, page_num):
    create_if_not_exists(part)
    full_file = page_formatted_file(part)

    with open(full_file, 'w') as f:
        f.write(f'{page_num},\n')

def start_page(part):
    create_if_not_exists(part)
    full_file = page_formatted_file(part)

    with open(full_file, 'r') as f:
        text = f.read()

    if text == '':
        return 1
    page_num = int(text.split(',\n')[-2].strip())

    return page_num + 1

In [ ]:
columns = ['Rank','Name','Platform','Year', \
        'Genre','Publisher','NA_Sales', 'EU_Sales',
        'JP_Sales', 'Other_Sales', 'Critic_Score', 'User_Score'
        ]

In [ ]:
offset = 1
part = 1
results_each_page = 200

csv_name_format = 'CSV_Files/vgsales_data_updated'
csv_file = f'{csv_name_format}_part_{part}.csv'

def scrape_pages(start, end):
    vg_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else None
    for page in range(start, end):
        batch_vg_data = scrape_vg_missing_data(page, results_each_page)
        if vg_data is None:
            vg_data = batch_vg_data
        else:
            vg_data = pd.concat([vg_data, batch_vg_data], ignore_index=True)

        vg_data.to_csv(csv_file, sep=",", encoding='utf-8', index=False)
        record_progress(part, page)

In [ ]:
import os

In [ ]:
# Arguments varirable for scraping
results_per_page = 200
total_pages = 100
start = start_page('')

page_start = start
page_end = total_pages

In [ ]:
print(f'starting at page {page_start} and ending in {page_end}')
scrape_pages(page_start, page_end, results_per_page)

starting at page 105 and ending in 100


In [ ]:
data_file = "vgsales_current.csv"
data = pd.read_csv(data_file, sep=",", encoding='utf-8')
print(data.shape)
data.head(5)

(17744, 10)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,32,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,29.02,3.77,8.51
1,52,Mario Kart 8 Deluxe,NS,2017.0,Racing,Nintendo,5.05,4.98,2.11,0.91
2,81,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77
3,87,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.91,12.92,3.80,3.35
4,94,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58


In [ ]:
missing_data = pd.read_csv('vgsales_critic_user.csv')
missing_data

,Rank,Name,Critic_Score,User_Score
0,32,Wii Sports,7.7,NaN
1,52,Mario Kart 8 Deluxe,9.3,NaN
2,81,Super Mario Bros.,10.0,8.2
3,87,Mario Kart Wii,8.2,9.1
4,94,Tetris,NaN,NaN
...,...,...,...,...
17731,20796,Brave: The Video Game,NaN,NaN
17732,20797,Pippa Funnell 2: Farm Adventures,NaN,NaN
17733,20798,Irotoridori no Sekai: World's End Re-Birth,NaN,NaN
17734,20799,PDC World Championship Darts 2009,NaN,NaN


In [ ]:
missing_data.drop('Name', axis=1, inplace=True)

In [ ]:
full_data = pd.merge(data, missing_data, on='Rank')
full_data

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,User_Score
0,32,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,29.02,3.77,8.51,7.7,NaN
1,52,Mario Kart 8 Deluxe,NS,2017.0,Racing,Nintendo,5.05,4.98,2.11,0.91,9.3,NaN
2,81,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,10.0,8.2
3,87,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.91,12.92,3.80,3.35,8.2,9.1
4,94,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17731,20796,Brave: The Video Game,DS,2012.0,Action-Adventure,Disney Interactive Studios,NaN,0.01,NaN,0.00,NaN,NaN
17732,20797,Pippa Funnell 2: Farm Adventures,DS,2007.0,Simulation,Ubisoft,NaN,0.01,NaN,0.00,NaN,NaN
17733,20798,Irotoridori no Sekai: World's End Re-Birth,PSV,2015.0,Action,HuneX,NaN,NaN,0.01,NaN,NaN,NaN
17734,20799,PDC World Championship Darts 2009,Wii,2009.0,Sports,Oxygen Interactive,NaN,0.01,NaN,0.00,NaN,NaN


In [ ]:
full_data.to_csv('vg_sales_full_data.csv', index=False)

In [ ]:
pd.read_csv('vg_sales_full_data.csv')

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,User_Score
0,32,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,29.02,3.77,8.51,7.7,NaN
1,52,Mario Kart 8 Deluxe,NS,2017.0,Racing,Nintendo,5.05,4.98,2.11,0.91,9.3,NaN
2,81,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,10.0,8.2
3,87,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.91,12.92,3.80,3.35,8.2,9.1
4,94,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17731,20796,Brave: The Video Game,DS,2012.0,Action-Adventure,Disney Interactive Studios,NaN,0.01,NaN,0.00,NaN,NaN
17732,20797,Pippa Funnell 2: Farm Adventures,DS,2007.0,Simulation,Ubisoft,NaN,0.01,NaN,0.00,NaN,NaN
17733,20798,Irotoridori no Sekai: World's End Re-Birth,PSV,2015.0,Action,HuneX,NaN,NaN,0.01,NaN,NaN,NaN
17734,20799,PDC World Championship Darts 2009,Wii,2009.0,Sports,Oxygen Interactive,NaN,0.01,NaN,0.00,NaN,NaN
